In [7]:
import numpy as np
import numpy.linalg as la
import scipy.linalg as sla
import numpy.random as rd

In [41]:
def l1eq_pd(x0, A, At, b, pdtol=1e-3, pdmaxiter=50, cgtol=1e-8, cgmaxiter=200):
    largescale = (str(type(A)) == "<class 'function'>")
    
    N = len(x0)
    alpha = 0.01
    beta = 0.5
    mu = 10
    
    gradf0 = np.zeros((N,2), float)
    gradf0[:,1] = 1
    
    if largescale:
        if la.norm(A(x0) - b)/la.norm(b) > cgtol:
            print("Starting point infeasible; using x0 = At*inv(AAt)*y")
            AAt = lambda z: A(At(z))
            w, cgres, cgiter = cgsolve(AAt, b, cgtol, cgmaxiter, 0)
            if cgres > 1/2:
                print("A*At is ill-conditioned: cannot find starting point")
                xp = x0
                return xp
            x0 = At(w)
    else:
        if la.norm(A.T.dot(x0) - b)/la.norm(b) > cgtol:
            print("Starting point infeasible; using x0 = At*inv(AAt)*y.")
            w, hcond = la.solve(A*A.conj().T, b)
            if hcond < 1e-14:
                print("A*At is ill-conditioned: cannot find starting point")
                xp = x0
                return xp
            x0 = A.conj().T*w
    x = x0
    u = (0.95)*abs(x0) + (0.10)*max(abs(x0))
    
    fu1 = x - u
    fu2 = -x - u
    lamu1 = -1/fu1
    lamu2 = -1/fu2
    if largescale:
        v = -A(lamu1-lamu2)
        Atv = At(v)
        rpri = A(x) - b
    else:
        v = -A*(lamu1-lamu2)
        Atv = A.conj().T.dot(v)
        rpri = A.T.dot(x) - b
    
    sdg = -(fu1.conj().T*lamu1 + fu2.conj().T*lamu2)
    tau = mu*2*N/sdg
    
    rcent = np.hstack((-lamu1*fu1, -lamu2*fu2)) - 1/tau
    rdual = gradf0 + np.hstack((lamu1-lamu2, -lamu1-lamu2)) + np.hstack((Atv, np.zeros(N,1)))
    resnorm = la.norm(np.hstack((rdual, rcent, rpri)))
    
    pditer = 0
    done = (sdg < pdtol) or (pditer >= pdmaxiter)
    
    while not done:
        pditer += 1
        
        w1 = -1/tau*(-1/fu1 + 1/fu2) - Atv
        w2 = -1 - 1/tau*(1/fu1 + 1/fu2)
        w3 = -rpri
        
        sig1 = -lamu1/fu1 - lamu2/fu2
        sig2 = lamu1/fu1 - lamu2/fu2
        sigx = sig1 - sig2**2/sig1
        
        if largescale:
            w1p = w3 - A(w1/sigx - w2*sig2/(sigx*sig1))
            h11pfun = lambda z: -A(1/sigx*At(z))
            dv, cgres, cgiter = cgsolve(h11pfun, w1p, cgtol, cgmaxiter, 0)
            if cgres > 1/2:
                print("Cannot solve system. Returning previous iterate.")
                xp = x
                return xp
            dx = (w1 - w2*sig2/sig1 - At(dv))/sigx
            Adx = A(dx)
            Atdv = At(dv)
        else:
            w1p = -(w3 - A*(w1/sigx - w2*sig2/(sigx*sig1)))
            H11p = A*(np.squeeze(np.diag(1/sigx))*A.conj().T)
            dv, hcond = la.solve(H11p, w1p)
            if hcond < 1e-14:
                print("Matrix ill-conditioned. Returning previous iterate.")
                xp = x
                return xp
            dx = (w1 - w2*sig2/sig1 - A.conj().T*dv)/sigx
            Adx = A*dx
            Atdv = A.conj().T*dv
        
        du = (w2 - sig2*dx)/sig1
        
        dlamu1 = (lamu1/fu1)*(-dx+du) - lamu1 - (1/tau)*1/fu1
        dlamu2 = (lamu2/fu2)*(dx+du) - lamu2 - 1/tau*1/fu2
        
        indp = np.transpose(np.nonzero(dlamu1 < 0))
        indn = np.transpose(np.nonzero(dlamu2 < 0))
        s = np.min(np.hstack((1, -lamu1[indp]/dlamu1[indp], -lamu2[indn]/dlamu2[indn])))
        indp = np.transpose(np.nonzero(dx - du) > 0)
        indn = np.transpose(np.nonzero(-dx - du) > 0)
        s = (0.99)*np.min(np.hstack((s, -fu1[indp]/(dx[indp] - du[indp]), -fu2[indn]/(-dx[indx]-du[indn]))))
        
        suffdec = 0
        backiter = 0
        while not suffdec:
            xp = x + s*dx
            up = u + s*du
            lamu1p = lamu1 + s*dlamu1
            lamu2p = lamu2 + s*dlamu2
            fu1p = xp - up
            fu2p = -xp - up
            rdp = gradf0 + np.hstack((lamu1p-lamu2p, -lamu1p-lamu2p)) + np.hstack((Atvp, np.zeros((N,1))))
            rcp = np.hstack((-lamu1p*fu1p, -lamu2p*fu2p)) - 1/tau
            rpp = rpri + s*Adx
            suffdec = (la.norm(np.hstack((rdp, rcp, rpp))) <= (1-alpha*s)*resnorm)
            s *= beta
            backiter += 1
            if backiter > 32:
                print("Stuck backtracking, returning last iterate.")
                xp = x
                return xp
            
        x = xp
        u = up
        v = vp
        Atv = Atvp
        lamu1 = lamu1p
        lamu2 = lamu2p
        fu1 = fu1p
        fu2 = fu2p
        
        sdg = -(fu1.conj().T*lamu1 + fu2.conj().T*lamu2)
        tau = mu*2*N/sdg
        rpri = rpp
        rcent = np.hstack((-lamu1*fu1, -lamu2*fu2)) - 1/tau
        rdual = gradf0 + np.hstack((lamu1-lamu2, -lamu1-lamu2)) + np.hstack((Atv, np.zeros((N,1))))
        resnorm = la.norm(np.hstack((rdual, rcent, rpri)))
        
        done = (sdg < pdtol) or (pditer >= pdmaxiter)
        
        print("Iteration = %d, tau = %8.3e, Primal = %8.3e, PDGap = %8.3e, Dual res = %8.3e, Primal res = %8.3e" \
              %(pditer, tau, np.sum(u), sdg, la.norm(rdual), la.norm(rpri)))
        if largescale:
            print("                CG Res = %8.3e, CG Iter = %d"%(cgres, cgiter))
        else:
            print("                H11p condition number = %8.3e"%(hcond))
            

def cgsolve(A, b, tol, maxiter, verbose=1):
    implicit = (str(type(A)) == "<class 'function'>")
    
    x = np.zeros((len(b), 1))
    r = b
    d = r
    delta = r.conj().T * r
    delta0 = b.conj().T * b
    numiter = 0
    bestx = x
    bestres = np.sqrt(delta/delta0)
    while ((numiter < maxiter) and (delta > tol**2*delta0)):
        if implicit:
            q = A(d)
        else:
            q = A*d
            
        alpha = delta/(d.conj().T * q)
        x += alpha*d

        if (numiter+1)%50 == 0:
            if implicit:
                r = b - A(x)
            else:
                r = b - A*x
        else:
            r = r - alpha*q

        deltaold = delta
        delta = r.conj().T * r
        beta = delta/deltaold
        d = r + beta*d
        numiter += 1
        if np.sqrt(delta/delta0) < bestres:
            bestx = x
            bestres = np.sqrt(delta/delta0)

        if verbose and numiter%verbose == 0:
            print("cg: Iter = %d, Best residual = %8.3e, Current residual = %8.3e"%(numiter, bestres, np.sqrt(delta/delta0)))
        
    if verbose:
        print("cg: Iterations = %d, best residual = %14.8e"%(numiter, bestres))
    
    x = bestx
    res = bestres
    Iter = numiter
    return x, res, Iter

In [42]:
N = 512
T = 20
K = 120

x = np.zeros((N,1))
q = rd.permutation(N)
x[q[:T]] = np.sign(rd.random((T,1)))

A = rd.random((K,N))
A = sla.orth(A.conj().T)

y = A.T.dot(x)
x0 = A.dot(y)

xp = l1eq_pd(x0, A, [], y, 1e-3)

ValueError: operands could not be broadcast together with shapes (512,2) (512,512) 